<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P2_slidewindow_for_machine_reading_comprehension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#机器阅读理解（QA）滑动窗口实现
#评估指标：精准匹配度（EM），模糊匹配度（F1）cmrc官方有一个eval函数，需要传入两个文件进行，使用cmrc之前需要下载相关库nltk:
# pip install nltk
# import nltk
# nltk.download("punkt")
# 也可以改写cmrc的eval文件，使之不需要传入两个文件，而是可以实时进行评估：https://github.com/zyds/transformers-code/blob/master/02-NLP%20Tasks/10-question_answering/cmrc_eval.py

In [2]:
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/MyDrive/nlp")
!pip install datasets
!pip install -U accelerate
!pip install -U transformers

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [3]:
from datasets import Dataset,load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, DefaultDataCollator, TrainingArguments, Trainer,pipeline

In [4]:
datasets=load_dataset("cmrc2018",cache_dir="./")
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
model=AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
datasets["train"][:5]
#这5个都是同一段context,但是问题和答案不一样

{'id': ['TRAIN_186_QUERY_0',
  'TRAIN_186_QUERY_1',
  'TRAIN_186_QUERY_2',
  'TRAIN_186_QUERY_3',
  'TRAIN_186_QUERY_4'],
 'context': ['范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
  '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；19

In [6]:
sample_data=datasets["train"].select(range(10))

In [7]:
#滑动窗口进行截断,stride就是重叠的长度
#truncation="only_second"代表只对text_pair=examples["context"]进行截断
tokenized_examples = tokenizer(text=sample_data["question"],
                               text_pair=sample_data["context"],
                               return_offsets_mapping=True,
                               return_overflowing_tokens=True,
                               stride=64,
                               max_length=384, truncation="only_second", padding="max_length")
tokenized_examples.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [8]:
print(tokenized_examples["overflow_to_sample_mapping"], len(tokenized_examples["overflow_to_sample_mapping"]),len(tokenized_examples["input_ids"]))
#本来只有10个data,但是由于有truncation和 stride，所以原来的一段话会被分割成几段，所以这里长度变成了25，overflowing代表每个部分属于原来的哪个id对应的内容
#tokenized_examples的所有key对应的value的长度都是25了
for sen in tokenizer.batch_decode(tokenized_examples["input_ids"][:3]):
    print(sen)
#这三句都是原来的'id'='TRAIN_186_QUERY_0'的内容
print(len(tokenized_examples["input_ids"][0]),len(tokenized_examples["offset_mapping"][0]))
tokenized_examples["offset_mapping"][:3]

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9] 25 25
[CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 范 廷 颂 枢 机 （ ， ） ， 圣 名 保 禄 · 若 瑟 （ ） ， 是 越 南 罗 马 天 主 教 枢 机 。 1963 年 被 任 为 主 教 ； 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理 ； 1994 年 被 擢 升 为 总 主 教 ， 同 年 年 底 被 擢 升 为 枢 机 ； 2009 年 2 月 离 世 。 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生 ； 童 年 时 接 受 良 好 教 育 后 ， 被 一 位 越 南 神 父 带 到 河 内 继 续 其 学 业 。 范 廷 颂 于 1940 年 在 河 内 大 修 道 院 完 成 神 学 学 业 。 范 廷 颂 于 1949 年 6 月 6 日 在 河 内 的 主 教 座 堂 晋 铎 ； 及 后 被 派 到 圣 女 小 德 兰 孤 儿 院 服 务 。 1950 年 代 ， 范 廷 颂 在 河 内 堂 区 创 建 移 民 接 待 中 心 以 收 容 到 河 内 避 战 的 难 民 。 1954 年 ， 法 越 战 争 结 束 ， 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1960 年 因 捍 卫 修 院 的 自 由 、 自 治 及 拒 绝 政 府 在 修 院 设 政 治 课 的 要 求 而 被 捕 。 1963 年 4 月 5 日 ， 教 宗 任 命 范 廷 颂 为 天 主 教 北 宁 教 区 主 教 ， 同 年 8 月 15 日 就 任 ； 其 牧 铭 为 「 我 信 [SEP]
[CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 由 、 自 治 及 拒 绝 政 府 在 修 院 设 政 治 课 的 要 求 而 被 捕 。 1963 年 4 月 5 日 

[[(0, 0),
  (0, 1),
  (1, 2),
  (2, 3),
  (3, 4),
  (4, 5),
  (5, 6),
  (6, 7),
  (7, 8),
  (8, 9),
  (9, 10),
  (10, 11),
  (11, 12),
  (12, 13),
  (13, 14),
  (14, 15),
  (0, 0),
  (0, 1),
  (1, 2),
  (2, 3),
  (3, 4),
  (4, 5),
  (5, 6),
  (6, 7),
  (7, 8),
  (8, 9),
  (9, 10),
  (10, 11),
  (11, 12),
  (12, 13),
  (13, 14),
  (14, 15),
  (15, 16),
  (16, 17),
  (17, 18),
  (18, 19),
  (19, 20),
  (20, 21),
  (21, 22),
  (22, 23),
  (23, 24),
  (24, 25),
  (25, 26),
  (26, 27),
  (27, 28),
  (28, 29),
  (29, 30),
  (30, 34),
  (34, 35),
  (35, 36),
  (36, 37),
  (37, 38),
  (38, 39),
  (39, 40),
  (40, 41),
  (41, 45),
  (45, 46),
  (46, 47),
  (47, 48),
  (48, 49),
  (49, 50),
  (50, 51),
  (51, 52),
  (52, 53),
  (53, 54),
  (54, 55),
  (55, 56),
  (56, 57),
  (57, 58),
  (58, 59),
  (59, 60),
  (60, 61),
  (61, 62),
  (62, 63),
  (63, 67),
  (67, 68),
  (68, 69),
  (69, 70),
  (70, 71),
  (71, 72),
  (72, 73),
  (73, 74),
  (74, 75),
  (75, 76),
  (76, 77),
  (77, 78),
  (78, 79)

In [9]:
print(tokenized_examples[0].sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [10]:
print(tokenized_examples.sequence_ids(0))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [11]:
print(tokenized_examples.sequence_ids(0).index(None,tokenized_examples[0].sequence_ids.index(1)))

383


In [12]:
def data_process(examples):
  tokenized_examples=tokenizer(examples["question"],
                               text_pair=examples["context"],
                               return_offsets_mapping=True,
                               return_overflowing_tokens=True,
                               stride=64,
                               max_length=256, truncation="only_second", padding="max_length")
  #因为要把每部分归还到原来的'id'对应的内容，所以遍历的是overflow_to_sample_mapping
  sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
  start_positions=[]
  end_positions=[]
  example_ids = [] #为了还原id

  #examples是以字典形式读取的，其中每个key对应的value都是一个list
  for i, _ in enumerate(sample_mapping):
    answer = examples["answers"][sample_mapping[i]]#这时候要找到原来的index就变成用sample_mapping[i]来找了
    str_start_index = answer['answer_start'][0] #'answers': {'text': ['1963年'], 'answer_start': [30]}}
    str_end_index = str_start_index + len(answer["text"][0])
    #从字符index找到token的index
    #首先找到context_token的起始位置,从context的起始位置逐渐逼近答案
    contexttokens_start_index=tokenized_examples[i].sequence_ids.index(1)
    contexttokens_end_index=tokenized_examples[i].sequence_ids.index(None,contexttokens_start_index)-1
    token2str = tokenized_examples["offset_mapping"][i]
    #首先把由于截断而使得答案不在留下的内容的example的答案定为0,0
    if token2str[contexttokens_end_index][1]<str_start_index or token2str[contexttokens_start_index][0]>str_end_index:
      answertoken_start_index=0
      answertoken_end_index=0
    else:
      answertoken_start_index=contexttokens_start_index
      while answertoken_start_index<=contexttokens_end_index and token2str[answertoken_start_index][0]<str_start_index:
        answertoken_start_index +=1
      answertoken_end_index=contexttokens_end_index
      while answertoken_end_index>=contexttokens_start_index and token2str[answertoken_end_index][1]>str_end_index:
        answertoken_end_index -=1
    start_positions.append(answertoken_start_index)
    end_positions.append(answertoken_end_index)
    example_ids.append(examples["id"][sample_mapping[i]])
    tokenized_examples["offset_mapping"][i] = [
            (o if tokenized_examples[i].sequence_ids[k] == 1 else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]
    #让问题部分、无意义的部分（就是sequence_ids为0或None的部分）的offset_mapping为None，这里是为了方便后面对validation和test部分的eval
  tokenized_examples["start_positions"] = start_positions
  tokenized_examples["end_positions"] = end_positions
  tokenized_examples["example_ids"] = example_ids
  return tokenized_examples



In [13]:
tokenied_datasets = datasets.map(data_process, batched=True, remove_columns=datasets["train"].column_names)
tokenied_datasets

Map:   0%|          | 0/162272 [00:00<?, ? examples/s]

Map:   0%|          | 0/51504 [00:00<?, ? examples/s]

Map:   0%|          | 0/32064 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_ids'],
        num_rows: 463408
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_ids'],
        num_rows: 149808
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_ids'],
        num_rows: 94176
    })
})

In [ ]:
print(tokenied_datasets["train"]["offset_mapping"][0])

In [ ]:
import collections
#创建一个字典，形式是 {example_ids(原ids)：[input_index]（在tokenized data里面的第index条）}
example2input = collections.defaultdict(list) #其实是example2tokenized_example
for i,example_id in enumerate(tokenied_datasets["train"]["example_ids"][:10]):
  example2input[example_id].append(i)
example2input


In [ ]:
import numpy as np
def get_results(start_index_logits,end_index_logits,examples,tokenized_examples):
  #start_index_logits是答案的开始位置索引的softmax计算结果;examples代表原数据；tokenized_examples就是tokenized以后的数据;
  #这个函数的目的是从answer index（从start到end）得到原文context中对应的内容
  predictions={}
  references={}

  example2input = collections.defaultdict(list) #其实是example2tokenized_example
  for i,example_id in enumerate(tokenized_examples["example_ids"]):
    example2input[example_id].append(i)

  #选取 top n 个作为候选答案
  n_top = 20
  #最大答案长度，
  # max_length=128
  for example in examples:
    example_id = example["id"]
    context = example["context"]
    answers = []
    for tokenized_example_id in example2input[example_id]:
      start_logits=start_index_logits[tokenized_example_id]
      end_logits=end_index_logits[tokenized_example_id]
      offset=tokenized_examples[tokenized_example_id]["offset_mapping"]
      start_indexs=np.argsort(start_logits)[::-1][:n_top].tolist()
      end_indexs=np.argsort(end_logits)[::-1][:n_top].tolist()
      for start_index in start_indexs:
        for end_index in end_indexs:
          if offset[start_index] is None or offset[end_index] is None:
            continue #这就是前面把所有非context的文本的offset_mapping设为None的意义所在
          # if end_index<start_index or end_index-start_index+1<max_length:
          if end_index<start_index:
            continue
          answers.append({
              "text":context[offset[start_index][0]:offset[end_index][1]],
              "score":start_logits[start_index]+end_logits[end_index]
          })
    if len(answers)>0:
      final_answer=max(answers,key=lambda x:x["score"])
      predictions[example_id]=final_answer["text"]
    else:
      predictions[example_id]=""
    references[example_id]=example["answers"]["text"]
  return predictions,references
from cmrc_eval import evaluate_cmrc
def metric(pred):
  start_logits,end_logits=pred[0]
  if start_logits.shape[0] == len(tokenied_datasets["validation"]):
    p, r = get_result(start_logits, end_logits, datasets["validation"], tokenied_datasets["validation"])
  else:
    p, r = get_result(start_logits, end_logits, datasets["test"], tokenied_datasets["test"])
  return evaluate_cmrc(p, r)



In [ ]:
args = TrainingArguments(
    output_dir="models_for_qa_window",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=20,
    save_steps=10,
    logging_steps=10,
    num_train_epochs=1
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenied_datasets["train"],
    eval_dataset=tokenied_datasets["validation"],
    data_collator=DefaultDataCollator()
    compute_metrics=metric
)

In [ ]:
trainer.train()

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/nlp/models_for_qa_window/checkpoint-210")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer,device=0)

pipe(question="今天是什么节日？", context="今天是国庆节。")